In [1]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from qdrant_client.http import models

In [2]:
qdrant_client = QdrantClient(
    url="",
    api_key="",
)

In [3]:
qdrant_client.recreate_collection(
    collection_name="mycollection",
    vectors_config=models.VectorParams(size=1024, distance=models.Distance.COSINE),
)
print("Create collection reponse:", qdrant_client)

/var/tmp/ipykernel_27686/1465433335.py:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


Create collection reponse: <qdrant_client.qdrant_client.QdrantClient object at 0x7f89682aba30>


In [4]:
collection_info = qdrant_client.get_collection(collection_name="mycollection")

print("Collection info:", collection_info)

Collection info: status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=0 points_count=0 segments_count=2 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=1024, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantiza

In [5]:
import pdfplumber

fulltext = ""
with pdfplumber.open("english_text.pdf") as pdf:
    # loop over all the pages
    for page in pdf.pages:
        fulltext += page.extract_text()

# print(fulltext)

In [6]:
text = fulltext

chunks = []
while len(text) > 500:
    last_period_index = text[:500].rfind('.')
    if last_period_index == -1:
        last_period_index = 500
    chunks.append(text[:last_period_index])
    text = text[last_period_index+1:]
chunks.append(text)


for chunk in chunks:
    print(chunk)
    print("---")

Good day Mr. Boss
I hope this email finds you well.
In this email I would like to propose a new idea concerning our way of working together as a
team.
Last weekend, I attended a TED talk as a guest. One of the speakers presented this new way of
working together, that is based on sharing more responsibility between team members.
The results of her studies are quite impressive
---
 The overall performance as well as the personal
well being and work-life balance of each member of the test group significantly improved over
the test periode. The whole staff, including the team lead, was wholeheartedly convinced by the
new method and the enterprise in question even launched an initiative to implement this new
way of doing things for the whole company
---

So what does this method include?
- flat hierarchies
- sharing responsibilities and including people during decision making
- open and blame-free environment
We have such a talented, diverse and motivated team and I am sure we could reach t

In [3]:
# Loading model
from sentence_transformers import SentenceTransformer


model = SentenceTransformer("dunzhang/stella_en_1.5B_v5", trust_remote_code=True, device='cuda:1')

/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [23]:
from qdrant_client.http.models import PointStruct



points = []
i = 1
for chunk in chunks:
    i += 1
    doc_embeddings = model.encode(chunk)
    print(doc_embeddings.shape)
    points.append(PointStruct(id=i, vector=doc_embeddings.tolist(), payload={"text": chunk}))

(1024,)
(1024,)
(1024,)
(1024,)
(1024,)


In [24]:
operation_info = qdrant_client.upsert(
    collection_name="mycollection",
    wait=True,
    points=points
)

print("Operation info:", operation_info)

Operation info: operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


# Chat functions

In [6]:
import ollama
def create_answer_with_context(query):
    query_embeddings = model.encode(query)

    search_result = qdrant_client.search(
        collection_name="mycollection",
        query_vector=query_embeddings, 
        limit=5
    )

    prompt = "Context:\n"
    for result in search_result:
        prompt += result.payload['text'] + "\n---\n"
    prompt += "Question:" + query + "\n---\n" + "Answer:"

    #print("----PROMPT START----")
    #print(":", prompt)
    #print("----PROMPT END----")
    
    
    completion = ollama.chat(model='llama3.1:8b', messages=[
  {
    'role': 'user',
    'content': prompt,
  },
])


    return completion["message"]["content"]

In [7]:
input = "what is the main idea of the new teams setup? "
answer = create_answer_with_context(input)
print(answer)

The main idea of the new team's setup is to adopt a flat hierarchy, share responsibilities among team members, and create an open and blame-free environment where everyone has a sense of ownership and stake in the process.
